### Dataset Description
The dataset provides comprehensive information on various socio-economic and environmental factors across 63 provinces in Vietnam by years (from 2015-2021)
1. ```Province (string):``` Represents information on the 63 provinces and municipalities in Vietnam.
2. ```Year:``` Indicates the year in which the data was collected.
3. ```AREA OF LAND (int):```Represents the total natural land area of each province per year (in thousand hectares).
4. ```POPULATION (float):``` Signifies the total population within the geographical area (in thousand persons).
5. ```POPULATION DENSITY (float):```Measures the population density in persons per square kilometer.
6. ```GROSS REGIONAL DOMESTIC PRODUCT (GRDP) (float):```Reflects the total value of goods and services produced in the economic region each year (in billion Vietnamese dong).
7. ```State Budget Revenue (float):```Represents the total revenue collected from taxes, fees, etc., by the province in the corresponding year (in billion Vietnamese dong).
8. ```State Budget Expenditure (float):```Indicates the total amount spent by the province on programs, projects, or other activities (in billion Vietnamese dong).
9. ```Investment at Current Prices (float):```Denotes the total capital invested by businesses, organizations, or the government in a specific region, measured at current values (in billion Vietnamese dong).
10. ```Number of Farms (int):```Represents the total count of farms in the region.
11. ```Planted Area of Cereals (float):```Indicates the total land area used for cultivating cereals such as wheat or other grains (in thousand hectares).
12. ```Production of Fishery (float):```Signifies the total quantity of fisheries produced in the region (in tons).
13. ```Index of Industrial Production (float):```Measures the growth or decline of industrial production activities in the region, expressed as a percentage.
14. ```Retail Sales of Goods at Current Prices (float):```Represents the total value of goods sold at current prices in retail activities within the region (in billion Vietnamese dong).
15. ```Number of Schools (int):``` the total number of educational institutions in the region.
16. ```Number of Medical Establishments (int):``` Represents the total count of medical centers in the region.
17. ```Carbon Gross Emissions(int):```Indicates the total amount of carbon emissions released into the environment in the region (in Megaton CO2e).
18. ```tc_los_ha - Tropical Forest Loss per Hectare (Target Variable) (int):```Represents the amount of tropical forest loss per hectare in the region.

### Streaming data merge
19. ``` temp(float) : ```  the current temperature in specific province 
20. ``` temp_min(float) :``` Minimum expected temperature 
21. ``` sunrise(float) :```  expected Time for sunrise 
22. ``` sunset(float) : ``` expected time for sunset  
23. ``` humidity(float) :``` humidity level in the air   
24. ```pressure(float) :``` atmospheric pressure 
25. ``` feels_like(float) :``` represent how the current temperature actually feels to a person 
26. ```temp_max(float) :``` maximum expected temperature
27. ```wind(float) :``` wind speed at the analyzed location 
28. ```Description(string): ``` weather condition description 

#### Import Libraries

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymongo
import os ,sys 

from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline

if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

#### Set up MongoDB Connection

In [13]:
database_username = "admin01"
database_password = "LZRKfNnjX38rCOMs"
database_cluster_name = "bigdataforengineering.wef2a2d.mongodb.net"
connection_string = f"mongodb+srv://{database_username}:{database_password}@{database_cluster_name}/?retryWrites=true&w=majority"

database_name = 'processed_data' 
client = pymongo.MongoClient(connection_string)

# Check if the connection was successful
if client.server_info():
    print("Connected to MongoDB successfully!")

db = client[database_name]

Connected to MongoDB successfully!


#### Get all data from 'processed_data' database in MongoDB Cluster

In [14]:
# Open Weather data
weather_collection = 'vie_weather'
weather_collection = db[weather_collection]
weather_cursor = weather_collection.find({})
weather_list = list(weather_cursor)
Vie_weather = pd.DataFrame(weather_list)

In [15]:
# Carbon forest estimation data
carbon_forest_collection = 'carbon_forest'
carbon_forest_collection = db[carbon_forest_collection]
cdf_cursor = carbon_forest_collection.find({})
carbon_deforest_list = list(cdf_cursor)
forest_carbon = pd.DataFrame(carbon_deforest_list) 

In [16]:
# Vietnam economic data
econ_collection = 'economic_indicators'
econ_collection = db[econ_collection]
econ_cursor = econ_collection.find({})
economic_list = list(econ_cursor)
economic = pd.DataFrame(economic_list)

In [17]:
# Standardize the 'province' column values to uppercase in 'economic', 'forest_carbon', and 'Vie_weather' DataFrames.
economic['province'] = economic['province'].str.upper()
forest_carbon['province'] = forest_carbon['province'].str.upper()
Vie_weather['province'] = Vie_weather['province'].str.upper()

In [18]:
if any(economic.index.duplicated()) or any(Vie_weather.index.duplicated()) or any(forest_carbon.index.duplicated()):
    raise ValueError("Duplicate values found in index.")

# Merge the DataFrames along axis 1
vndata = pd.concat([forest_carbon, Vie_weather, economic], axis=1)
vndata.columns

Index(['_id', 'carbon_gross_emissions', 'tc_loss_ha', 'year', 'province',
       '_id', 'temp', 'temp_min', 'sunrise', 'sunset', 'description',
       'humidity', 'pressure', 'feels_like', 'temp_max', 'wind', 'province',
       '_id', 'province', 'year', 'AREA OF LAND (Thous. ha)',
       'POPULATION (Thous. pers.)', 'POPULATION DENSITY (Person/km2)',
       'At current prices (Bill. dongs)', 'State budget revenue (Bill. dongs)',
       'State budget expenditure (Bill. dongs)',
       'Investment at current prices (Bill. dongs)', 'Number of farms',
       'Planted area of cereals (Thous. ha)', 'Production of fishery (Ton)',
       'Index of industrial production (%)',
       'Retail sales of goods at current prices (Bill. dongs)',
       'Number of schools (School)',
       'Number of medical establishments (Esta.)'],
      dtype='object')

In [19]:
def dataframe_cleaning(dataframe):
    # Drop id column
    dataframe = dataframe.drop(['_id'], axis=1)
    
    # Drop duplicate columns
    dataframe = dataframe.loc[:, ~dataframe.columns.duplicated(keep='first')]
    
    # Rename columns 
    dataframe.rename(columns={'AREA OF LAND (Thous. ha)': 'AREA OF LAND','POPULATION (Thous. pers.)':'Population', 'POPULATION DENSITY (Person/km2)':'population density', 'At current prices (Bill. dongs)':'GROSS REGIONAL DOMESTIC PRODUCT', 
                       'State budget revenue (Bill. dongs)':'STATE BUDGET REVENUE', 'State budget expenditure (Bill. dongs)':'STATE BUDGET EXPENDITURE',
                       'Investment at current prices (Bill. dongs)': 'INVESTMENT AT CURRENT PRICES',
                       'Number of farms': 'NUMBER OF FARM', 'Planted area of cereals (Thous. ha)': 'PLANTED AREA OF CEREALS',
                       'Production of fishery (Ton)':'PRODUCTION OF FISHERY','Index of industrial production (%)': 'INDEX OF INDUSTRIAL PRODUCTION',
                       'Retail sales of goods at current prices (Bill. dongs)': 'RETAIL SALES OF GOODS', 'Number of schools (School)':'NUMBER OF SCHOOLS',
                       'Number of medical establishments (Esta.)': 'NUMBER OF MEDICAL ESTABLISHMENTS',
                       'carbon_gross_emissions': 'CARBON GROSS EMISSIONS', 'tc_loss_ha':'TROPICAL FOREST LOSS',
                       'FEELS_LIKE': 'FEELS LIKE','TEMP_MIN': 'TEMP MIN','TEMP_MAX': 'TEMP MAX'}, inplace=True)
    
    # Uppercase all columns name
    dataframe.columns = dataframe.columns.str.upper()

    # Fill the NaN values by the mean in each province
    # The retail sail of goods data for Thanh Hoa and Quang tri is missing, so I will fill them by the mean of the country

    # List of columns to fill NaN values
    columns_to_fill = ['AREA OF LAND', 'POPULATION DENSITY', 'GROSS REGIONAL DOMESTIC PRODUCT',
                    'STATE BUDGET REVENUE', 'STATE BUDGET EXPENDITURE', 'NUMBER OF FARM',
                    'RETAIL SALES OF GOODS', 'NUMBER OF SCHOOLS']

    # Calculate mean values for each column grouped by province
    mean_values_by_province = dataframe.groupby('PROVINCE')[columns_to_fill].mean()

    # Iterate through each column and fill NaN values with the mean of the respective province
    for col in columns_to_fill:
        # Use lambda function to round the values and fill NaN with the mean of the province
        dataframe[col] = dataframe.apply(lambda row: round(row[col], 2) if pd.notnull(row[col]) else round(mean_values_by_province.loc[row['PROVINCE'], col], 2), axis=1)

    # Fill NaN values in 'RETAIL SALES OF GOODS' with the mean of the entire column
    dataframe['RETAIL SALES OF GOODS'].fillna(dataframe['RETAIL SALES OF GOODS'].mean(), inplace=True)

    
    return dataframe

In [20]:
# Wrap the cleaning function in a FunctionTransformer
cleaning_transformer = FunctionTransformer(dataframe_cleaning)

# Define the cleaning pipeline
cleaning_pipeline = Pipeline([
    ('cleaning', cleaning_transformer),
])

data_cleaned = cleaning_pipeline.fit_transform(vndata)


In [24]:
def save_data(dataframe, excel_name="default", mongo_collection_name='default'):
    # Save the cleaned data into an Excel file
    dataframe.to_excel(f"../data/processed/{excel_name}.xlsx", index=False)
    
    # Save the cleaned data into MongoDB Cluster
    records = dataframe.to_dict('records')
    collection = db[mongo_collection_name]

    # Reset data in collection
    if mongo_collection_name in db.list_collection_names():
        # Collection exists, so drop it
        db[mongo_collection_name].drop()
        print(f"Reset collection {mongo_collection_name}")
    else:
        print(f"Collection {mongo_collection_name} does not exist")

    collection.insert_many(records)
    
    # Check if cleaned data was inserted into MongoDB collection
    cursor = collection.find({})
    data_list = list(cursor)
    updated_data = pd.DataFrame(data_list)
    print("Data successfully inserted into MongoDB collection:", mongo_collection_name)

    return updated_data

In [26]:
# Call function to save cleaned data
save_data(dataframe=data_cleaned, 
                  excel_name="vndata_cleaned",
                  mongo_collection_name="vndata_cleaned"
                  )

Collection vndata_cleaned does not exist
Data successfully inserted into MongoDB collection: vndata_cleaned


,_id,CARBON GROSS EMISSIONS,TROPICAL FOREST LOSS,YEAR,PROVINCE,TEMP,TEMP_MIN,SUNRISE,SUNSET,DESCRIPTION,...,STATE BUDGET REVENUE,STATE BUDGET EXPENDITURE,INVESTMENT AT CURRENT PRICES,NUMBER OF FARM,PLANTED AREA OF CEREALS,PRODUCTION OF FISHERY,INDEX OF INDUSTRIAL PRODUCTION,RETAIL SALES OF GOODS,NUMBER OF SCHOOLS,NUMBER OF MEDICAL ESTABLISHMENTS
0,65a9e7cccb25423c2530cf61,24733,229,2015,AN GIANG,298.12,298.12,1705447219,1705488927,scattered clouds,...,13141.9,17687.6,11228.7,697.0,652.8,348079.0,105.0,35835.0,547.00,191
1,65a9e7cccb25423c2530cf62,110738,904,2016,AN GIANG,295.80,295.80,1705015152,1705056774,broken clouds,...,14282.0,18002.0,11920.3,1180.0,677.1,369843.0,105.7,38093.0,542.00,192
2,65a9e7cccb25423c2530cf63,200469,930,2017,AN GIANG,303.12,303.12,1705360808,1705402497,broken clouds,...,16165.1,22522.4,13488.6,1063.0,649.4,401724.0,106.8,43129.0,536.00,3560
3,65a9e7cccb25423c2530cf64,126319,540,2018,AN GIANG,302.12,302.12,1705533630,1705575357,broken clouds,...,19303.5,24880.1,16146.9,1016.0,630.6,486804.0,108.9,47437.0,530.00,3807
4,65a9e7cccb25423c2530cf65,34024,174,2019,AN GIANG,297.12,297.12,1705015152,1705056774,overcast clouds,...,20539.3,26701.5,17695.7,966.0,632.2,539909.0,109.9,51421.0,531.00,3969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,65a9e7cccb25423c2530d115,360933,752,2017,YEN BAI,290.92,290.92,1705016457,1705055733,overcast clouds,...,14616.7,14413.5,10370.8,16.0,70.7,7497.0,106.9,12665.0,192.00,358
437,65a9e7cccb25423c2530d116,226560,450,2018,YEN BAI,292.90,292.90,1705534871,1705574380,overcast clouds,...,16345.4,16025.7,11105.3,32.0,70.6,9017.0,104.3,13868.0,197.00,371
438,65a9e7cccb25423c2530d117,12688,63,2019,YEN BAI,296.89,296.89,1705448472,1705487939,overcast clouds,...,18736.1,18480.1,13513.1,34.0,71.6,10485.0,110.9,15979.0,179.00,393
439,65a9e7cccb25423c2530d118,970,8,2020,YEN BAI,292.97,292.97,1705534871,1705574380,overcast clouds,...,20577.5,20472.8,15655.2,17.0,72.3,11634.0,108.1,16728.0,177.00,409


In [66]:
def categorical_encode(dataframe):
    # Extract the 'DESCRIPTION' column from the DataFrame
    categorical = dataframe['DESCRIPTION']

    # Initialize a LabelEncoder
    le = LabelEncoder()

    # Transform the categorical values into numerical labels using LabelEncoder
    categorical = le.fit_transform(categorical)

    # Create a new DataFrame with the transformed numerical labels, named 'DESCRIPTION'
    categorical = pd.DataFrame(categorical, columns=['DESCRIPTION'])

    # Concatenate the original DataFrame and the new 'DESCRIPTION' DataFrame along the columns
    dataframe = pd.concat([dataframe, categorical], axis=1)

    return dataframe

In [67]:
def handle_outliers_iqr(column):
    # Calculate the first quartile (Q1)
    Q1 = column.quantile(0.25)
    
    # Calculate the third quartile (Q3)
    Q3 = column.quantile(0.75)
    
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Create a copy of the column to avoid modifying the original data
    column_copy = column.copy()
    
    # Replace values below the lower bound with the lower bound
    column_copy.loc[column < lower_bound] = lower_bound
    
    # Replace values above the upper bound with the upper bound
    column_copy.loc[column > upper_bound] = upper_bound
    
    # Return the column with outliers handled
    return column_copy


In [68]:
def dataframe_transforming(dataframe):
    # Encode categorical features
    categorical = categorical_encode(dataframe)

    # Select numerical features excluding 'year'
    numerical_features = dataframe.select_dtypes(include=['int64', 'float64']).columns
    numerical_features = numerical_features[numerical_features != 'year']
    
    # Extract the numerical features from the original DataFrame
    vn_feature = dataframe[numerical_features]

    # Create a copy of the numerical features to avoid modifying the original data
    vn_features_copy = vn_feature.copy()

    # Handle outliers in each numerical column using the handle_outliers_iqr function
    for col in vn_features_copy.columns:
        vn_features_copy[col] = handle_outliers_iqr(vn_features_copy[col])

    # Update the numerical features with the outliers handled
    vn_feature = vn_features_copy

    # Concatenate the numerical features and the encoded categorical features along the columns
    vn_feature = pd.concat([vn_feature, categorical], axis=1)

    # Return the transformed DataFrame
    return vn_feature


In [69]:
# Wrap the transform function in a FunctionTransformer
feature_transformer = FunctionTransformer(dataframe_transforming)

# Define the transform pipeline
feature_pipeline = Pipeline([
    ('transform', feature_transformer),
])

data_feature = feature_pipeline.fit_transform(data_cleanned)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17608\1270085861.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-16209624.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  column_copy.loc[column < lower_bound] = lower_bound
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17608\1270085861.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-71479.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  column_copy.loc[column < lower_bound] = lower_bound
C:\Users\ASUS\AppData\Local\Temp\ipykernel_17608\1270085861.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1004.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  column_copy.loc[column < lower_bound] = low

In [70]:
# Call function to save transformed data
save_data(dataframe=data_feature, 
                  excel_name="vndata_transformed",
                  mongo_collection_name="vndata_transformed"
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17608\2045940017.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  records = dataframe.to_dict('records')


Reset collection vndata_transformed
Data successfully inserted into MongoDB collection: vndata_transformed


,_id,CARBON GROSS EMISSIONS,TROPICAL FOREST LOSS,YEAR,TEMP,TEMP_MIN,SUNRISE,SUNSET,HUMIDITY,PRESSURE,...,INVESTMENT AT CURRENT PRICES,NUMBER OF FARM,PLANTED AREA OF CEREALS,PRODUCTION OF FISHERY,INDEX OF INDUSTRIAL PRODUCTION,RETAIL SALES OF GOODS,NUMBER OF SCHOOLS,NUMBER OF MEDICAL ESTABLISHMENTS,PROVINCE,DESCRIPTION
0,65a9639ef8f774b35acdd11e,24733,229,2015,298.12,298.12,1705447219,1705488927,77,1013,...,11228.7,697.0,652.8,348079.0,105.0,35835.0,547.00,191,AN GIANG,6
1,65a9639ef8f774b35acdd11f,110738,904,2016,295.80,295.80,1705015152,1705056774,86,1011,...,11920.3,1180.0,677.1,369843.0,105.7,38093.0,542.00,192,AN GIANG,0
2,65a9639ef8f774b35acdd120,200469,930,2017,303.12,303.12,1705360808,1705402497,49,1011,...,13488.6,1063.0,649.4,401724.0,106.8,43129.0,536.00,3560,AN GIANG,0
3,65a9639ef8f774b35acdd121,126319,540,2018,302.12,302.12,1705533630,1705575357,62,1013,...,16146.9,1016.0,630.6,486804.0,108.9,47437.0,530.00,3807,AN GIANG,0
4,65a9639ef8f774b35acdd122,34024,174,2019,297.12,297.12,1705015152,1705056774,90,1011,...,17695.7,966.0,632.2,539909.0,109.9,51421.0,531.00,3969,AN GIANG,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,65a9639ef8f774b35acdd2d2,360933,752,2017,290.92,290.92,1705016457,1705055733,97,1017,...,10370.8,16.0,70.7,7497.0,106.9,12665.0,192.00,358,YEN BAI,5
437,65a9639ef8f774b35acdd2d3,226560,450,2018,292.90,292.90,1705534871,1705574380,95,1015,...,11105.3,32.0,70.6,9017.0,104.3,13868.0,197.00,371,YEN BAI,5
438,65a9639ef8f774b35acdd2d4,12688,63,2019,296.89,296.89,1705448472,1705487939,69,1015,...,13513.1,34.0,71.6,10485.0,110.9,15979.0,179.00,393,YEN BAI,5
439,65a9639ef8f774b35acdd2d5,970,8,2020,292.97,292.97,1705534871,1705574380,95,1017,...,15655.2,17.0,72.3,11634.0,108.1,16728.0,177.00,409,YEN BAI,5
